<a href="https://colab.research.google.com/github/xinyanz-erin/Applied-Finance-Project/blob/Lilian/European_jax_random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://colab.chainer.org/install |sh -
import cupy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   6610      0 --:--:-- --:--:-- --:--:--  6610
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 58.9 MB 21 kB/s 
     |████████████████████████████████| 1.0 MB 41.0 MB/s 
+ set +ex
Installation succeeded!


In [2]:
# Peter's code

def simple_process(key, initial_stocks, numsteps, drift, cov):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key, drift, cov, (numsteps+1,))
    sigma = jnp.diag(cov) ** 0.5
    def time_step(t, val):
        dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) + noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalue(key, initial_stocks, numsteps, drift, cov, strike):
  return jnp.mean((jnp.maximum(batch_simple(keys, initial_stocks, numsteps, drift, cov)[:,-1,:]-strike,0))) # this is assuming 1 stock for testing price (didn't take avg)

def optionvalueavg(key, initial_stocks, numsteps, drift, cov, strike):
  return jnp.mean((jnp.maximum(jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov)[:,-1,:], axis=1)-strike,0))) # this is assuming 3 stocks in basket

In [3]:
# Peter's code

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

numstocks = 3

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
#numsteps = 10
drift = jnp.array([0.0]*numstocks)

cov = jnp.identity(numstocks)*.25*.25
initial_stocks = jnp.array([100.]*numstocks)

K = 110.0

fast_simple = jax.jit(simple_process, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, 1000000)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None))

print(initial_stocks) #S = 100
print(K) #K = 110
print(cov) #sigma = 0.25
print(drift) #drift = 0

#################################################################################### values for checking
#S, K, r, sigma, T
# 100, 110, 0, 0.25, 1
# 1 stock price should be around 6.1904
# 3 stock price should be around 2.3767
# delta should be around (0.39888 / numstocks)
####################################################################################

# option price
# 1 stock
print(optionvalue(key, initial_stocks, 1, drift, cov, K)) # numsteps here = years = 1
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, 1, drift, cov, K)) # numsteps here = years = 1

# delta test
gooptionvalue = jax.grad(optionvalue,argnums=1)
gooptionvalue(keys, initial_stocks, 1, drift, cov, K) # numsteps here = years = 1

[100. 100. 100.]
110.0
[[0.0625 0.     0.    ]
 [0.     0.0625 0.    ]
 [0.     0.     0.0625]]
[0. 0. 0.]
6.195752
2.3685584


DeviceArray([0.13313732, 0.13304195, 0.13283277], dtype=float32)

In [46]:
def simple_process(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key, drift, cov, (numsteps+1,))
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalue(key, initial_stocks, numsteps, drift, cov, strike, T):
  return jnp.mean((jnp.maximum(batch_simple(keys, initial_stocks, numsteps, drift, cov, T)[:,-1,:]-strike,0))) # this is assuming 1 stock for testing price (didn't take avg)

def optionvalueavg(key, initial_stocks, numsteps, drift, cov, strike, T):
  return jnp.mean((jnp.maximum(jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T)[:,-1,:], axis=1)-strike,0))) # this is assuming 3 stocks in basket

numstocks = 3

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 50
drift = jnp.array([0.0]*numstocks)

cov = jnp.identity(numstocks)*.25*.25
initial_stocks = jnp.array([100.]*numstocks)

T = 1.0
K = 110.0

fast_simple = jax.jit(simple_process, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, 1000000)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# print(initial_stocks) #S = 100
# print(K) #K = 110
# print(cov) #sigma = 0.25
# print(drift) #drift = 0

#################################################################################### values for checking
#S, K, r, sigma, T
# 100, 110, 0, 0.25, 1
# 1 stock price should be around 6.1904
# 3 stock price should be around 2.3767
# delta should be around (0.39888 / numstocks)
####################################################################################

# option price
# 1 stock
# print(optionvalue(key, initial_stocks, numsteps, drift, cov, K, T)) # numsteps here = 50, but T = year = 1
# 3 stocks basket
# print(optionvalueavg(key, initial_stocks, numsteps, drift, cov, K, T)) # numsteps here = 50, but T = year = 1

# delta test
gooptionvalue = jax.grad(optionvalue,argnums=1)
gooptionvalue(keys, initial_stocks, numsteps, drift, cov, K, T) # numsteps here = 50, but T = year = 1

# Random Test

In [88]:
numstocks = 2
numsteps = 50
numpath = 3

drift = jnp.array([0.0]*numstocks)
cov = jnp.identity(numstocks)*.25*.25

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
keys = jax.random.split(key, numpath)
random_num = jnp.zeros((numpath,numsteps+1,numstocks))

indx = 0
for k in keys:
  new_random_num = jax.random.multivariate_normal(k, drift, cov, (numsteps+1,))
  random_num = jax.ops.index_update(random_num,
                  jax.ops.index[indx],
                            new_random_num)
  indx += 1
#  print(jax.random.multivariate_normal(k, drift, cov, (numsteps+1,)))
print(random_num)

[[[ 1.06821172e-01  2.21046224e-01]
  [ 2.40737826e-01 -1.62713930e-01]
  [-7.98098743e-02 -1.55660138e-01]
  [ 9.91226137e-02  1.75049201e-01]
  [ 1.04640070e-02 -1.03172280e-01]
  [ 1.10001229e-02  3.11474979e-01]
  [ 2.99181752e-02 -4.27707136e-01]
  [-7.73551986e-02 -2.00432435e-01]
  [-2.43155003e-01 -2.37499341e-01]
  [-8.06378275e-02 -1.04600489e-01]
  [-1.28517762e-01  1.88006997e-01]
  [-2.43509606e-01  2.41018981e-01]
  [ 1.98988929e-01  1.10572025e-01]
  [-2.65934151e-02  3.75110172e-02]
  [-4.91731942e-01 -3.63588423e-01]
  [-1.61849439e-01 -2.38929525e-01]
  [-2.08552461e-02  4.19329524e-01]
  [ 1.26985937e-01  4.51336861e-01]
  [-5.58958277e-02  8.52403119e-02]
  [ 2.29354039e-01 -2.37519637e-01]
  [-2.53097620e-02 -7.52949715e-02]
  [-1.28073350e-01 -3.92606109e-01]
  [-5.24904072e-01  1.93697050e-01]
  [-3.29366356e-01 -3.25284392e-01]
  [-2.69997362e-02 -5.80900535e-02]
  [-3.50705206e-01  1.27057686e-01]
  [ 1.38078764e-01  1.10236675e-01]
  [-7.27485180e-01 -2.046973

In [81]:
jnp.zeros((numsteps+1,numstocks))

DeviceArray([[0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.]], dtype=float32)

In [77]:
int(1)

1